## 데이터 수집

### 주제에 대한 배경:
- 여론조사 기간인 http://www.realmeter.net 에서 커피전무점 선호도
  1위-스타벅스,  2위-이디아, 3위-빽다방 였다(2018년도)
- 최근 젊은 세대 사이에는 신 주거 트렌드를 반영한 '스세권'이라는 부동산
  용어가 유행할 만큼 스타벅스의 인기와 영향력은 높다.
- 이번실습에서는 대한민국의 가장 큰 도시 서울에서 스타벅스가 어떤 전략으로
  매장 입지를 선택하는지 살펴본다.

### 데이터 분석을 통해 확인 할 가설:

1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.

### 1. 데이터 수집 - 스타벅스 매장위치

 - 스타벅스 입지전략을 분석하기 위해 서울시 내에 출점한 스타벅스 위치를
   파악한다.
 - 두 가설을 검증하기 위해 인구통계 데이터 수집과 스타벅스 홈페이지에서
   스타벅스 매장들의의 정보를 크롤링으로 통해 수집한다.
 - 서울시 열린데이터 광장 OPEN API를 이용하여 인구통계 데이터
   (거주인구 수, 직장인구 수)를 수집한다.(OPEN API: excel파일로 데이터를 다운받지않고 python code로 프로그램내에서 사용하도록 데이터를 받아온다)

다음과 같은 조건으로 서울시 스타벅스 매장 목록 검색:

http://www.istarbucks.co.kr 접속 -> [매장찾기] -> [지역검색] -> [서울] -> [전체]

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd

In [2]:
browser = webdriver.Chrome('c:/Study/Python/datadown/chromedriver.exe')
url='http://www.istarbucks.co.kr '
browser.get(url)

In [3]:
#스타벅스 홈페이지에서 webdriver로 '매장찾기' -> '지역 검색' 버튼을 클릭해서 이동 
browser.find_element_by_xpath('/html/body/div[3]/div[6]/div/div/div/nav/ul/li[5]/a').click()
browser.find_element_by_xpath('/html/body/div[4]/div[7]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a').click()

In [4]:
#webdriver로 ‘서울’ 버튼 요소를 찾아 클릭
seoul_btn='#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(seoul_btn).click()

In [5]:
#webdriver로 '전체' 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(all_btn).click()

In [6]:
#BeautifulSoup으로 HTML 파서 만들기
html = browser.page_source
soup = BeautifulSoup(html,'html.parser')

- 여기까지 실행하면 웹 브라우저에 서울시 전체 스타벅스 매장 목록이 왼쪽에 나타난다.
- 검색결과로 545개의 스타벅스 매장명칭, 주소, 전화번호등이 나타난다.


- select 메서드로 요소를 가져오기위해 검색결과 페이지의 구성와 html 태그의 구성을 살펴본다. 

- 다음과 같이 검색 결과가 나열되어있다:
        - 검색결과 item들 각각은 li 요소 < li 요소들을 묶음으로 갖고있는 ul요소
- 각 ul, li요소들의 class 이름을 사용해서 태그를 찾을 수 있도록 지정한다 ("태그 요소 이름.class 이름" format으로 표기한다. 

In [7]:
#select()를 이용해 원하는 HTML 태그를 모두 찾아오기
sb_all = soup.select('ul.quickSearchResultBoxSidoGugun>li.quickResultLstCon')
print(len(sb_all))
sb_all[0]

545


<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [24]:
sb_store = sb_all[0]

In [25]:
name = sb_store['data-name'].strip()
name

'역삼아레나빌딩'

In [26]:
lat = sb_store['data-lat'].strip()
lat

'37.501087'

In [27]:
long = sb_store['data-long'].strip()
long

'127.043069'

In [30]:
addr = str(sb_store.select('p.result_details')[0])
addr

'<p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p>'

In [32]:
address = addr.split('<br/>')[0].split('>')[1]
address

'서울특별시 강남구 언주로 425 (역삼동)'

In [38]:
phone = addr.split('<br/>')[1].split('<')[0]
phone

'1522-3232'

In [41]:
store_type = sb_store.select('i')[0]['class'][0][4:]
store_type #일반매장은 general, 리져브매장은 reserve로 표기되어있음.

'general'

**매장목록 데이터 만들기**

- BeautifulSoup을 통해서 가져온 HTML 의 내용을 반복문을 통해서 각 매장별로
  매장정보(매장명,위/경도, 매장타입, 주소, 전화번호)를 추출하고 sb_list
  변수에 담는다.
- sb_list 변수의 값을 컬럼을 생성하여 seoul_sb_df 데이터프레임에 담는다.

In [8]:
sb_list=[]
for x in sb_all:
    name = x['data-name'].strip()
    lat = x['data-lat'].strip()
    long = x['data-long'].strip()
    addr = str(x.select('p.result_details')[0])
    address = addr.split('<br/>')[0].split('>')[1]
    phone = addr.split('<br/>')[1].split('<')[0]
    stype = x.select('i')[0]['class'][0][4:]
    sb_list.append([name, lat,long, address, phone, stype])

In [9]:
cols =['매장명','lat','long','주소','전화번호','매장타입']
seoul_sb_df=pd.DataFrame(sb_list, columns=cols)
seoul_sb_df.head()

,매장명,lat,long,주소,전화번호,매장타입
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,general
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,general
2,신사역성일빌딩,37.514132,127.020563,서울특별시 강남구 강남대로 584 (논현동),1522-3232,general
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,general
4,스탈릿대치R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,reserve


In [10]:
#데이터프레임의 요약 정보 확인
seoul_sb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     545 non-null    object
 1   lat     545 non-null    object
 2   long    545 non-null    object
 3   주소      545 non-null    object
 4   전화번호    545 non-null    object
 5   매장타입    545 non-null    object
dtypes: object(6)
memory usage: 25.7+ KB


- 데이터프레임을 엑셀로 저장한다.
- files 폴더 생성

In [53]:
#엑셀로 저장
seoul_sb_df.to_excel('../data3/seoul_sb_list.xlsx',index=False)

In [11]:
browser.close()

### 2. 공공 데이터 수집 - OPEN API활용 

- 정보,지자체 등 공공기관은 보건,교육,인구/가구,안전,환경 등 다양한 공공부분에서
  업무를 통해 만들어낸 수많은 공공데이터를 기업체와 시민이 이용,접근,재사용할 수
  있도록 개방한다
- 서울특별시는 서울열린데이터광장(https://data.seoul.go.kr/together/guide/useGuide.do) 사이트를 통해 시민에게
  서울 행정구역 내의 다양한 공공데이터를 제공한다.
- 사용자들은 해당 사이트에서 데이터를 내려받거나 OPEN API를 통해서 획득한다.
- 실습을 위해 서울열린데이터광장에서 OPEN API를 사용해서 시군구 목록, 주민등록
  인구 통계, 사업체 수 정보를 수집한다.

#### 참고:

**서울열린데이터광장 일반인증키 발급받기:**

- 서울시 홈페이지에서 통합회원가입후 
  서울열린데이터광장(https://data.seoul.go.kr/together/guide/useGuide.do) 
  페이지 하단의 -> 일반 인증키 발급 신청 클릭한다.
- 신청화면에서 신청양식에 적절한 내용을 입력한다. 
- 서울열린데이터를 이용해 개발중인 웹 페이지가 없다면 'http://'만 입력한다.
- 신청완료되면 [나의화면]에서 [인증키 관리]에서 인증키를 확인한다.
  https://data.seoul.go.kr/together/mypage/actkeyMng_ss.do
- 해당 인증키를 최근90일 중 30일을 초과해서 사용할 수 없고, 한번에 1000건이 
  넘는 데이트를 조회하는데 사용할 수 없다. 1000건이 넘을 경우 여러번 나눠서
  호출 방식으로 모든 데이터를 조회할 수 있다

#### requests 라이브러리 활용

requests is a Python module that you can use to send all kinds of HTTP requests. It is an easy-to-use library with a lot of features ranging from passing parameters in URLs to sending custom headers and SSL Verification

참고 link: https://www.edureka.co/blog/python-requests-tutorial/


In [65]:
#requests 라이브러리
import requests
req = requests.get("https://www.stackoverflow.com/")
print(req.encoding)
print(req.status_code)
print(req.headers['Content-Type'])

utf-8
200
text/html; charset=utf-8


### 서울열린데이터 OPEN API 호출하기 실습

- OPEN API에서는 사용자가 요청하는 조건들을 URL주소에 포함해야한다.

http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{OBJECTID}

- KEY :  인증키
- TYPE : 결과데이터타입(JSON/XML ..등)
- SERVICE : 해당 API 서비스명(SdeTlSccoSigW - 서울시 행정구역 시군구 정보)
- START_INDEX , END_INDEX : 가져오고 싶은 데이터의 시작번호와 종료번호를 의미한다.
- OBJECT : 필수가 아니기기때문에 별도로 값을 입력하지 않는다.

In [12]:
##서울열린데이터광장에서 시군구 목록을 가져오는 OPEN API를 호출하는 코드:

#라이브러리 임포트 (위에서 import했으면, 주석처리)
import requests
import pandas as pd 

#자주 사용하는 변수를 미리 선언하고 인증키 저장
SEOUL_API_AUTH_KEY = '4b6b586d616c696e3937634f554c61'
 
#호출을 위한 URL 변수 선언
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(SEOUL_API_AUTH_KEY)
print(url)
        
result_dict = requests.get(url).json()
#result_dict #총 25개의 데이터 갯수를 가진 dictionary

http://openapi.seoul.go.kr:8088/4b6b586d616c696e3937634f554c61/json/SdeTlSccoSigW/1/25


**json 데이터 --> dataframe 변환**

json의 결과값을 딕셔너리 데이터값으로 가져와서 데이터 분석이 용이한 데이터프레임으로 변환한다.

In [13]:
result_data = result_dict['SdeTlSccoSigW']
#result_data

In [14]:
print(result_data['list_total_count'])
print(result_data['RESULT'])
print(result_data['row'][0])

25
{'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}
{'OBJECTID': 1.0, 'SIG_CD': '11320', 'SIG_KOR_NM': '도봉구', 'SIG_ENG_NM': 'Dobong-gu', 'ESRI_PK': 0.0, 'LAT': '37.6658609', 'LNG': '127.0317674'}


In [15]:
data_list = result_data['row']
#data_list

In [16]:
#data_list의 데이터를 담을 DataFrame 생성
sample_df = pd.DataFrame(data_list)
print(sample_df.info())
sample_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   OBJECTID    25 non-null     float64
 1   SIG_CD      25 non-null     object 
 2   SIG_KOR_NM  25 non-null     object 
 3   SIG_ENG_NM  25 non-null     object 
 4   ESRI_PK     25 non-null     float64
 5   LAT         25 non-null     object 
 6   LNG         25 non-null     object 
dtypes: float64(2), object(5)
memory usage: 1.5+ KB
None


,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546
5,6.0,11530,구로구,Guro-gu,5.0,37.4954856,126.858121
6,7.0,11110,종로구,Jongno-gu,6.0,37.5990998,126.9861493
7,8.0,11305,강북구,Gangbuk-gu,7.0,37.6469954,127.0147158
8,9.0,11260,중랑구,Jungnang-gu,8.0,37.5953795,127.0939669
9,10.0,11680,강남구,Gangnam-gu,9.0,37.4959854,127.0664091


#### 서울열린데이터광장 OPEN API를 호출하는 함수 작성
- 함수에 OPEN API 호출 URL과 요청 서비스 키를 전달해 결과 리스트를 받는
  함수를 작성한다.

In [17]:
#API호출 key
SEOUL_API_AUTH_KEY = '4b6b586d616c696e3937634f554c61'

#서울열린데이터광장 OPEN API호출 공통 함수
def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list

In [18]:
sgg_url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(SEOUL_API_AUTH_KEY)
sgg_data_list = seoul_open_api_data(url, 'SdeTlSccoSigW')
sgg_data_list[0]

{'OBJECTID': 1.0,
 'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0.0,
 'LAT': '37.6658609',
 'LNG': '127.0317674'}

결과데이터 데이터프레임으로 변환하고 프레임의 컬럼명을 변경한다.
결과 데이터를 pandas의 데이터프레임으로 변환

In [19]:
cols = ['SIG_CD','SIG_KOR_NM','LAT','LNG']
sgg_df = pd.DataFrame(data=sgg_data_list, columns=cols)
sgg_df.head()

,SIG_CD,SIG_KOR_NM,LAT,LNG
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [20]:
sgg_df.columns= ['시군구코드', '시군구명', '위도', '경도']
sgg_df.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [21]:
sgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구코드   25 non-null     object
 1   시군구명    25 non-null     object
 2   위도      25 non-null     object
 3   경도      25 non-null     object
dtypes: object(4)
memory usage: 928.0+ bytes


In [85]:
#엑셀로 저장
sgg_df.to_excel('../data3/seoul_sgg_list.xlsx', index=False)

### 가설 1을 검증하기위한 데이터 수집:

#### 공통 함수를 이용해 서울시 시군구별 인구 데이터 가져오기

- 인구통계 데이터를 수집하기 위해 서울열린데이터광장 사이트에서
  '서울시 주민등록인구 (구별) 통계'라는 이름으로 공공데이터를 검색
- 검색된 결과에서 Sheet 버튼을 클릭한다.

In [22]:
sgg_pop_df = pd.read_csv('../data3/report_인구.txt', sep='\t', encoding='utf-8', skiprows=[1,2])
print(sgg_pop_df.info())
#sgg_pop_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기간        26 non-null     int64  
 1   자치구       26 non-null     object 
 2   세대        26 non-null     object 
 3   인구        26 non-null     object 
 4   인구.1      26 non-null     object 
 5   인구.2      26 non-null     object 
 6   인구.3      26 non-null     object 
 7   인구.4      26 non-null     object 
 8   인구.5      26 non-null     object 
 9   인구.6      26 non-null     object 
 10  인구.7      26 non-null     object 
 11  인구.8      26 non-null     object 
 12  세대당인구     26 non-null     float64
 13  65세이상고령자  26 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory usage: 3.0+ KB
None


In [23]:
condition = sgg_pop_df['자치구']!='합계'
sgg_pop_df_filter =sgg_pop_df[condition]
sgg_pop_df_filter

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
1,2019,종로구,"73,947","161,869","78,215","83,654","151,290","73,746","77,544","10,579","4,469","6,110",2.05,"27,519"
2,2019,중구,"62,739","136,488","66,823","69,665","126,171","61,910","64,261","10,317","4,913","5,404",2.01,"23,275"
3,2019,용산구,"110,126","245,185","119,599","125,586","228,670","110,356","118,314","16,515","9,243","7,272",2.08,"38,791"
4,2019,성동구,"135,838","308,979","150,861","158,118","300,889","147,273","153,616","8,090","3,588","4,502",2.22,"43,947"
5,2019,광진구,"164,428","366,972","176,917","190,055","351,350","170,262","181,088","15,622","6,655","8,967",2.14,"47,902"
6,2019,동대문구,"164,191","363,023","178,490","184,533","346,194","172,113","174,081","16,829","6,377","10,452",2.11,"59,350"
7,2019,중랑구,"181,579","402,024","199,006","203,018","397,015","196,959","200,056","5,009","2,047","2,962",2.19,"65,164"
8,2019,성북구,"192,592","454,744","219,099","235,645","442,650","214,437","228,213","12,094","4,662","7,432",2.30,"70,850"
9,2019,강북구,"144,277","317,695","154,638","163,057","313,954","153,235","160,719","3,741","1,403","2,338",2.18,"60,577"
10,2019,도봉구,"138,508","335,631","163,961","171,670","333,362","163,051","170,311","2,269",910,"1,359",2.41,"58,714"


In [24]:
sgg_pop_df_filter.info() #맨윗 row 합계를 제외한 25개 rows - 25개의 자치구가 있다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기간        25 non-null     int64  
 1   자치구       25 non-null     object 
 2   세대        25 non-null     object 
 3   인구        25 non-null     object 
 4   인구.1      25 non-null     object 
 5   인구.2      25 non-null     object 
 6   인구.3      25 non-null     object 
 7   인구.4      25 non-null     object 
 8   인구.5      25 non-null     object 
 9   인구.6      25 non-null     object 
 10  인구.7      25 non-null     object 
 11  인구.8      25 non-null     object 
 12  세대당인구     25 non-null     float64
 13  65세이상고령자  25 non-null     object 
dtypes: float64(1), int64(1), object(12)
memory usage: 2.9+ KB


#### 필요한 columns만 추출

구와 구별 인구수만 필요하기 때문에 자치구, 인구열만 필요하다

In [25]:
columns = ['자치구','인구']
sgg_pop_df_final = sgg_pop_df_filter[columns]
sgg_pop_df_final

,자치구,인구
1,종로구,"161,869"
2,중구,"136,488"
3,용산구,"245,185"
4,성동구,"308,979"
5,광진구,"366,972"
6,동대문구,"363,023"
7,중랑구,"402,024"
8,성북구,"454,744"
9,강북구,"317,695"
10,도봉구,"335,631"


In [109]:
#column이름 수정
sgg_pop_df_final.columns=['시군구명','주민증록인구']
sgg_pop_df_final

,시군구명,주민증록인구
1,종로구,"161,869"
2,중구,"136,488"
3,용산구,"245,185"
4,성동구,"308,979"
5,광진구,"366,972"
6,동대문구,"363,023"
7,중랑구,"402,024"
8,성북구,"454,744"
9,강북구,"317,695"
10,도봉구,"335,631"


In [110]:
#엑셀로 저장
sgg_pop_df_final.to_excel('../data3/sgg_pop.xlsx', index=False)

### 가설 2를 검증하기 위한 데이터 수집

#### 서울시 시군구별 사업체 데이터 가져오기

- 서울열린데이터광장에서 '서울시 사업체현황 (산업대분류별/동별) 통계'
  를 검색하여 Sheet 클릭하여 소수점 0, 기간 2019~2019 선택후 조회누루고
  내려받기(txt) 클릭하여 데이터를 받는다.
- 탭으로 구분되어 있는 데이터에서 1,2행은 무시하고 3행부터 읽어온다.

In [26]:
sgg_biz_df = pd.read_csv('../data3/report_사업체.txt', sep='\t', encoding='utf-8', skiprows=[1,2])
sgg_biz_df.head()

,기간,자치구,동,합계,합계.1,합계.2,합계.3,합계.4,농업 임업 및 어업,농업 임업 및 어업.1,...,공공행정 국방 및 사회보장 행정,공공행정 국방 및 사회보장 행정.1,교육 서비스업,교육 서비스업.1,보건업 및 사회복지 서비스업,보건업 및 사회복지 서비스업.1,예술 스포츠 및 여가관련 서비스업,예술 스포츠 및 여가관련 서비스업.1,협회 및 단체 수리 및 기타 개인 서비스업,협회 및 단체 수리 및 기타 개인 서비스업.1
0,2019,합계,합계,"823,624","281,318","5,226,997","2,885,614","2,341,383",24,426,...,"1,296","141,582","35,761","351,320","29,168","400,143","23,527","92,752","69,345","189,557"
1,2019,종로구,소계,"39,679","13,356","260,446","146,392","114,054",3,18,...,94,"15,109",829,"13,474",750,"17,366",899,"6,034","2,439","7,649"
2,2019,종로구,사직동,"3,574","1,203","47,522","25,154","22,368",1,12,...,23,"6,088",83,786,106,"1,096",91,"1,653",253,906
3,2019,종로구,삼청동,732,327,"4,601","2,271","2,330",-,-,...,4,"1,291",17,297,7,30,19,185,25,194
4,2019,종로구,부암동,599,255,"3,454","1,764","1,690",-,-,...,3,60,38,"1,190",19,183,18,49,82,325


In [27]:
condition=sgg_biz_df['동']=='소계'
sgg_biz_df_f1=sgg_biz_df[condition]
#sgg_biz_df_f1

In [28]:
columns=['자치구','합계','합계.2']
sgg_biz_df_final=sgg_biz_df_f1[columns]
sgg_biz_df_final

,자치구,합계,합계.2
1,종로구,"39,679","260,446"
19,중구,"60,127","392,568"
35,용산구,"20,254","135,240"
52,성동구,"28,343","178,243"
70,광진구,"24,445","126,305"
86,동대문구,"31,324","138,400"
101,중랑구,"28,228","103,345"
118,성북구,"23,617","111,005"
139,강북구,"19,014","73,904"
153,도봉구,"18,628","72,939"


In [29]:
sgg_biz_df_final.columns=['시군구명','사업체수','종사자수']
sgg_biz_df_final

,시군구명,사업체수,종사자수
1,종로구,"39,679","260,446"
19,중구,"60,127","392,568"
35,용산구,"20,254","135,240"
52,성동구,"28,343","178,243"
70,광진구,"24,445","126,305"
86,동대문구,"31,324","138,400"
101,중랑구,"28,228","103,345"
118,성북구,"23,617","111,005"
139,강북구,"19,014","73,904"
153,도봉구,"18,628","72,939"


In [30]:
#불필요한 rows 제외 후, row의 인덱스 번호 재정비
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final

,시군구명,사업체수,종사자수
0,종로구,"39,679","260,446"
1,중구,"60,127","392,568"
2,용산구,"20,254","135,240"
3,성동구,"28,343","178,243"
4,광진구,"24,445","126,305"
5,동대문구,"31,324","138,400"
6,중랑구,"28,228","103,345"
7,성북구,"23,617","111,005"
8,강북구,"19,014","73,904"
9,도봉구,"18,628","72,939"


In [104]:
#excel파일로 저장
sgg_biz_df_final.to_excel('../data3/sgg_biz.xlsx', index=False)